# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather_data = pd.read_csv("../output_data/cities.csv")
city_weather_data

,Unnamed: 0,City,Cloudiness (%),Country,Date,Humidity (%),Latitude,Longitude,Maximum Temperature,Wind Speed (mph)
0,0,Saint-André-les-Vergers,0,FR,1621891032,76,48.2853,4.0521,48.24,16.11
1,1,Hasaki,22,JP,1621890549,81,35.7333,140.8333,63.00,7.38
2,2,Saint-Philippe,85,RE,1621890352,78,-21.3585,55.7679,71.19,13.71
3,3,Khatanga,100,RU,1621891032,99,71.9667,102.5000,31.51,2.35
4,4,Mataura,6,NZ,1621890531,93,-46.1927,168.8643,32.25,3.04
...,...,...,...,...,...,...,...,...,...,...
538,538,Boa Vista,75,BR,1621891158,74,2.8197,-60.6733,82.38,5.75
539,539,Fortuna,1,US,1621891159,64,40.5982,-124.1573,74.43,4.00
540,540,Mazatlán,1,MX,1621891159,66,23.2167,-106.4167,89.56,7.00
541,541,Taltal,100,CL,1621890629,68,-25.4000,-70.4833,66.54,5.17


In [3]:
weather_data = city_weather_data.drop(columns=["Unnamed: 0"])
weather_data

,City,Cloudiness (%),Country,Date,Humidity (%),Latitude,Longitude,Maximum Temperature,Wind Speed (mph)
0,Saint-André-les-Vergers,0,FR,1621891032,76,48.2853,4.0521,48.24,16.11
1,Hasaki,22,JP,1621890549,81,35.7333,140.8333,63.00,7.38
2,Saint-Philippe,85,RE,1621890352,78,-21.3585,55.7679,71.19,13.71
3,Khatanga,100,RU,1621891032,99,71.9667,102.5000,31.51,2.35
4,Mataura,6,NZ,1621890531,93,-46.1927,168.8643,32.25,3.04
...,...,...,...,...,...,...,...,...,...
538,Boa Vista,75,BR,1621891158,74,2.8197,-60.6733,82.38,5.75
539,Fortuna,1,US,1621891159,64,40.5982,-124.1573,74.43,4.00
540,Mazatlán,1,MX,1621891159,66,23.2167,-106.4167,89.56,7.00
541,Taltal,100,CL,1621890629,68,-25.4000,-70.4833,66.54,5.17


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure Gmaps
gmaps.configure(api_key=g_key)

In [5]:
# Use the Lat and Lng as locations and Humidity as the weight.
locations = weather_data[['Latitude', 'Longitude']].astype(float)
humidity = weather_data['Humidity (%)'].astype(int)

In [7]:
# Plot heat map
fig = gmaps.figure(zoom_level=1, center=(0,0))

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=100,
                                point_radius=3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
ideal_conditions = weather_data.loc[(weather_data['Humidity (%)'] < 60) \
                                         & (weather_data['Wind Speed (mph)'] < 8) \
                                         & (weather_data['Cloudiness (%)'] < 25) \
                                         & (weather_data['Maximum Temperature'] >= 75) \
                                         & (weather_data['Maximum Temperature'] <= 88)].dropna()
ideal_conditions

,City,Cloudiness (%),Country,Date,Humidity (%),Latitude,Longitude,Maximum Temperature,Wind Speed (mph)
95,Los Altos,1,US,1621891045,48,37.3852,-122.1141,86.50,3.00
104,Saint Anthony,1,US,1621891054,42,45.0205,-93.2180,86.81,1.99
282,Nizwá,1,OM,1621891093,36,22.9333,57.5333,86.77,2.19
330,Mehrān,0,IR,1621891105,13,33.1222,46.1646,80.08,7.09
369,Gouré,4,NE,1621891114,9,13.9835,10.2704,87.69,7.87
372,Jalu,0,LY,1621891114,11,29.0331,21.5482,81.52,7.25
428,Hazorasp,3,UZ,1621891130,18,41.3194,61.0742,76.84,7.14
451,Rome,1,US,1621891136,38,43.2128,-75.4557,76.86,5.75
476,Tabas,0,IR,1621891021,22,33.5959,56.9244,83.98,4.16
482,Conway,1,US,1621890883,56,35.0887,-92.4421,85.98,4.00


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# create new dataframe with desired columns
hotel_df = pd.DataFrame(ideal_conditions[['City', 'Country', 'Latitude', 'Longitude']].copy())
# add column for hotel name
hotel_df['Hotel Name'] = ""
# reset index to reflect index numbers in new df
hotel_df = hotel_df.reset_index()
del hotel_df['index']
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
0,Los Altos,US,37.3852,-122.1141,
1,Saint Anthony,US,45.0205,-93.2180,
2,Nizwá,OM,22.9333,57.5333,
3,Mehrān,IR,33.1222,46.1646,
4,Gouré,NE,13.9835,10.2704,
5,Jalu,LY,29.0331,21.5482,
6,Hazorasp,UZ,41.3194,61.0742,
7,Rome,US,43.2128,-75.4557,
8,Tabas,IR,33.5959,56.9244,
9,Conway,US,35.0887,-92.4421,


In [10]:
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lon = row["Longitude"]
    coords =  f"{lat},{lon}"
    
    params = {"location": coords, "radius": 5000, "types": "lodging", "key": g_key}


    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    print(f"Results for Index {index}: {row['City']}")
    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    # convert response to json-----used next 2 lines of code to see error when google api billing was not enabled
#     places_data = response.json()

#     # Print the json (pretty printed)
#     print(json.dumps(places_data, indent=4, sort_keys=True))

    # get results
    results = response['results']
    
    # add results to hotel_df 
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
    except IndexError:
        print(f"No results found for: {row['City']}")
        
hotel_df

Results for Index 0: Los Altos
Results for Index 1: Saint Anthony
Results for Index 2: Nizwá
Results for Index 3: Mehrān
Results for Index 4: Gouré
No results found for: Gouré
Results for Index 5: Jalu
Results for Index 6: Hazorasp
Results for Index 7: Rome
Results for Index 8: Tabas
Results for Index 9: Conway
Results for Index 10: Corinth
Results for Index 11: Jalesar


,City,Country,Latitude,Longitude,Hotel Name
0,Los Altos,US,37.3852,-122.1141,Residence Inn by Marriott Palo Alto Mountain View
1,Saint Anthony,US,45.0205,-93.2180,"Holiday Inn Express Roseville-St. Paul, an IHG..."
2,Nizwá,OM,22.9333,57.5333,Orient Apartments
3,Mehrān,IR,33.1222,46.1646,بوتیک زنانه بهار
4,Gouré,NE,13.9835,10.2704,
5,Jalu,LY,29.0331,21.5482,صاحبة الفخامة
6,Hazorasp,UZ,41.3194,61.0742,Kemping wifi hotel
7,Rome,US,43.2128,-75.4557,Wingate By Wyndham Rome
8,Tabas,IR,33.5959,56.9244,شرکت معماری مهراز شایگان آراد
9,Conway,US,35.0887,-92.4421,"Country Inn & Suites by Radisson, Conway, AR"


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [21]:
# Add marker layer ontop of heat map--Banking Desert HeatMap Activity week 3
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,)
markers = gmaps.marker_layer(locations)
fig = gmaps.figure(zoom_level=1, center=(0,0))
fig.add_layer(hotel_layer)
fig.add_layer(markers)
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:

# following code came from tutoring session:
# target_coordinates = [{"lat": hotel_df['Latitude'], "lon": hotel_df['Longitude']}]
# target_coordinates
# target=[]
# for item in hotel_df.iterrows():
#     target.append({"lat":item[1]['Latitude'],"lon":item[1]['Longitude']})
# print(target)

# target_coordinates=[]
# for item in hotel_df.iterrows():
#     target_coordinates.append([item[1]['Latitude'],item[1]['Longitude']])
# print(target_coordinates)
